In [1]:
# Downloading Dependencies
import pandas as pd
import numpy as np 
import nltk
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AMAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv(r"C:\Users\AMAN\Downloads\DATA Scientist\GitHub\NLP Learning\all_kindle_review.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
data = data[['reviewText', 'rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
data.shape

(12000, 2)

In [5]:
# Checking for missing values
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
# Checking for Unique values in rating column
data['rating'].unique()

array([3, 5, 4, 2, 1])

In [7]:
# Checking for distribution of values in rating column
data['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [8]:
# Data Preprocessing and Cleaning
# Positive Review is 1 and negative review is 0
data['rating'] = data['rating'].apply(lambda x:0 if x<3 else 1)

In [9]:
data['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [10]:
# Converting text to lowercase
data['reviewText'] = data['reviewText'].str.lower()
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [11]:
%pip install lxml

# Removing Special Characters
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '',x))
data['reviewText'] = data['reviewText'].apply(lambda x:" ".join(y for y in x.split() if y not in stopwords.words('english')))
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftpl|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))
data['reviewText'] = data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join(x.split()))

Note: you may need to restart the kernel to use updated packages.


In [12]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [13]:
# Train Test Split
from sklearn.model_selection import train_test_split
x = data['reviewText']
y = data['rating']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_train_bow = cv.fit_transform(x_train).toarray()
x_test_bow = cv.transform(x_test).toarray()

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x_train_tfidf = tfidf.fit_transform(x_train).toarray()
x_test_tfidf = tfidf.transform(x_test).toarray()

In [17]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
nb_model_bow = mnb.fit(x_train_bow, y_train)
nb_model_tfidf = mnb.fit(x_train_tfidf, y_train)

In [20]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred_bow = nb_model_bow.predict(x_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(x_test_tfidf)

In [22]:
print("Bow Accuracy Score: ", accuracy_score(y_test, y_pred_bow)*100)
print("Tfidf Accuracy Score: ", accuracy_score(y_test, y_pred_tfidf)*100)

Bow Accuracy Score:  70.45833333333333
Tfidf Accuracy Score:  69.25


In [23]:
confusion_matrix(y_test, y_pred_bow)

array([[  98,  705],
       [   4, 1593]])

In [24]:
confusion_matrix(y_test, y_pred_tfidf)

array([[  65,  738],
       [   0, 1597]])